In [598]:
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt


In [599]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [600]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, LancasterStemmer, PorterStemmer
from nltk import word_tokenize, pos_tag
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [601]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


pd.set_option('display.max_columns', None)

from gensim.models import Word2Vec


from matplotlib import pyplot as plt
import seaborn as sns


from plotly.subplots import make_subplots
import plotly.graph_objects as go

#Use fivethirtyeight style
plt.style.use('fivethirtyeight')

#Cancel any style and reset matplotlib styles
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]


In [602]:
df = pd.read_csv("C:/Users/dimit/OneDrive/Desktop/git_projects/Recommendation System/Recommendation-System/n_movies.csv")
df

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413"
...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,After an experimental gene therapy turns them ...,"['Morgan Taylor Campbell, ', 'Italia Ricci, ',...","3,130"
9953,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a com...,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melis...","970,067"
9954,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,898"
9955,Supernatural,(2005–2020),TV-14,44 min,"Drama, Fantasy, Horror",8.4,Two brothers follow their father's footsteps a...,"['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim ...","439,601"


In [603]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9957 entries, 0 to 9956
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        9957 non-null   object 
 1   year         9430 non-null   object 
 2   certificate  6504 non-null   object 
 3   duration     7921 non-null   object 
 4   genre        9884 non-null   object 
 5   rating       8784 non-null   float64
 6   description  9957 non-null   object 
 7   stars        9957 non-null   object 
 8   votes        8784 non-null   object 
dtypes: float64(1), object(8)
memory usage: 700.2+ KB


In [604]:
df.drop_duplicates(subset=['title'], keep='first', inplace=True)

In [605]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7912 entries, 0 to 9912
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        7912 non-null   object 
 1   year         7410 non-null   object 
 2   certificate  4737 non-null   object 
 3   duration     6459 non-null   object 
 4   genre        7848 non-null   object 
 5   rating       6872 non-null   float64
 6   description  7912 non-null   object 
 7   stars        7912 non-null   object 
 8   votes        6872 non-null   object 
dtypes: float64(1), object(8)
memory usage: 618.1+ KB


In [606]:
df['title'] = df['title'].str.replace(' ', '_')

In [607]:
df.duration = df.duration.str.replace('min', '')
df.duration.fillna(0, inplace=True) 
df.duration = df.duration.astype(int)
df.duration.replace(0, df.duration.mean(), inplace=True)

In [608]:
df['votes'] = df['votes'].str.replace(',', '')

# Convert the column to float
df['votes'] = df['votes'].astype(float)

In [609]:
genres = df['genre'].str.split(', ', expand=True)
df = pd.concat([df, genres], axis=1)

In [610]:
def clean_stars(stars_string):
    cleaned_string = stars_string.strip("[]").replace("'", "").replace(", ", ",").split(",")
    return list(map(str.strip, cleaned_string))

# Apply the cleaning function to the stars column
df['stars'] = df['stars'].map(clean_stars)


def clean_stars(stars_list):
    return [name for name in stars_list if name]  # Keep only non-empty strings

# Apply the cleaning function to the stars column
df['stars'] = df['stars'].apply(clean_stars)

In [611]:
stars_expanded = df['stars'].apply(pd.Series)

C:\Users\dimit\AppData\Local\Temp\ipykernel_12552\3418863794.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stars_expanded = df['stars'].apply(pd.Series)


In [612]:
stars_expanded = stars_expanded.iloc[:, :3]
stars_expanded.rename(columns={0: 'Alpha', 1: 'Beta', 2: 'Gamma'}, inplace=True)

In [613]:
df = pd.concat([df, stars_expanded], axis=1)

In [614]:
words = stopwords.words("english")

In [615]:
lemmatizer = WordNetLemmatizer()

In [616]:
df['description'] = df['description'].apply(lambda x: " ".join([lemmatizer.lemmatize(i) for i in re.sub("[^a-zA-Z]", " ", str(x)).split() if i not in words]).lower())

In [617]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dimit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [618]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dimit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [619]:
df['tokenized'] = df['description'].apply(word_tokenize)

In [620]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\dimit\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [621]:
df['POS'] = df['tokenized'].apply(pos_tag)

In [622]:
# tag_filter = ["NN","NNS","NNP","NNPS"]  #keep only nouns, adjectives and adverbs
tag_filter = ["NN","NNS","NNP","NNPS","RB","RBR","RBS","JJ","JJR","JJS"] 
# filter pos function
def pos_filter(row):
    keep_list = []
    for x,y in row:
        if y in tag_filter:
            keep_list.append(x)
    return keep_list
df['clean'] = df['POS'].apply(pos_filter)

In [623]:
df.head(3)

,title,year,certificate,duration,genre,rating,description,stars,votes,0,1,2,Alpha,Beta,Gamma,tokenized,POS,clean
0,Cobra_Kai,(2018– ),TV-14,30.0,"Action, Comedy, Drama",8.5,decades all valley karate tournament bout midd...,"[Ralph Macchio, William Zabka, Courtney Hengge...",177031.0,Action,Comedy,Drama,Ralph Macchio,William Zabka,Courtney Henggeler,"[decades, all, valley, karate, tournament, bou...","[(decades, NNS), (all, DT), (valley, VBP), (ka...","[decades, tournament, middle, daniel, larusso,..."
1,The_Crown,(2016– ),TV-MA,58.0,"Biography, Drama, History",8.7,follows political rivalry romance queen elizab...,"[Claire Foy, Olivia Colman, Imelda Staunton, M...",199885.0,Biography,Drama,History,Claire Foy,Olivia Colman,Imelda Staunton,"[follows, political, rivalry, romance, queen, ...","[(follows, VBZ), (political, JJ), (rivalry, NN...","[political, rivalry, romance, queen, ii, reign..."
2,Better_Call_Saul,(2015–2022),TV-MA,46.0,"Crime, Drama",8.9,the trial tribulation criminal lawyer jimmy mc...,"[Bob Odenkirk, Rhea Seehorn, Jonathan Banks, P...",501384.0,Crime,Drama,None,Bob Odenkirk,Rhea Seehorn,Jonathan Banks,"[the, trial, tribulation, criminal, lawyer, ji...","[(the, DT), (trial, NN), (tribulation, NN), (c...","[trial, tribulation, criminal, lawyer, jimmy, ..."


In [624]:
def put_title_first(row):
    title = row['title']
    description = row['clean']
    if isinstance(description, list):
        description.insert(0, title)  # change it if use word2vec
    else:
        description = [title, description]
    return description


def put_genre1(row):
    genre1 = row[0]
    description = row['clean']
    if isinstance(description, list):
        description.insert(0, genre1)
    else:
        description = [genre1, description]
    return description

def put_genre2(row):
    genre2 = row[1]
    description = row['clean']
    if isinstance(description, list):
        description.insert(1, genre2)
    else:
        description = [genre2, description]
    return description

def put_genre3(row):
    genre3 = row[2]
    description = row['clean']
    if isinstance(description, list):
        description.insert(2, genre3)
    else:
        description = [genre3, description]
    return description


def put_stars(row):
    names = row['Alpha']
    description = row['clean']
    if isinstance(description, list):
        description.insert(2, names)
    else:
        description = [names, description]
    return description

def put_co_star(row):
    names = row['Beta']
    description = row['clean']
    if isinstance(description, list):
        description.insert(3, names)
    else:
        description = [names, description]
    return description

In [625]:
# df['movie_description'] = df.apply(put_title_first, axis=1)
df['movie_description'] = df.apply(put_genre1, axis=1)
df['movie_description'] = df.apply(put_genre2, axis=1)
# df['movie_description'] = df.apply(put_genre3, axis=1)
df['movie_description'] = df.apply(put_stars, axis=1)
# df['movie_description'] = df.apply(put_co_star, axis=1)

In [626]:
df.reset_index(drop=True, inplace=True)


In [627]:
df.head(3)

,title,year,certificate,duration,genre,rating,description,stars,votes,0,1,2,Alpha,Beta,Gamma,tokenized,POS,clean,movie_description
0,Cobra_Kai,(2018– ),TV-14,30.0,"Action, Comedy, Drama",8.5,decades all valley karate tournament bout midd...,"[Ralph Macchio, William Zabka, Courtney Hengge...",177031.0,Action,Comedy,Drama,Ralph Macchio,William Zabka,Courtney Henggeler,"[decades, all, valley, karate, tournament, bou...","[(decades, NNS), (all, DT), (valley, VBP), (ka...","[Action, Comedy, Ralph Macchio, decades, tourn...","[Action, Comedy, Ralph Macchio, decades, tourn..."
1,The_Crown,(2016– ),TV-MA,58.0,"Biography, Drama, History",8.7,follows political rivalry romance queen elizab...,"[Claire Foy, Olivia Colman, Imelda Staunton, M...",199885.0,Biography,Drama,History,Claire Foy,Olivia Colman,Imelda Staunton,"[follows, political, rivalry, romance, queen, ...","[(follows, VBZ), (political, JJ), (rivalry, NN...","[Biography, Drama, Claire Foy, political, riva...","[Biography, Drama, Claire Foy, political, riva..."
2,Better_Call_Saul,(2015–2022),TV-MA,46.0,"Crime, Drama",8.9,the trial tribulation criminal lawyer jimmy mc...,"[Bob Odenkirk, Rhea Seehorn, Jonathan Banks, P...",501384.0,Crime,Drama,None,Bob Odenkirk,Rhea Seehorn,Jonathan Banks,"[the, trial, tribulation, criminal, lawyer, ji...","[(the, DT), (trial, NN), (tribulation, NN), (c...","[Crime, Drama, Bob Odenkirk, trial, tribulatio...","[Crime, Drama, Bob Odenkirk, trial, tribulatio..."


In [628]:
genres_no_movies_no_series = ['Game-Show', 'Reality-TV', 'News', 'Talk-Show']

# Create a new DataFrame containing only the rows with the specified genres
reality = df[df['genre'].isin(genres_no_movies_no_series)]

In [629]:
df = df[~df['genre'].isin(genres_no_movies_no_series)]

In [630]:
df['year'] = df['year'].fillna('')
series = df[df['year'].str.contains('–')]
movies = df[~df['year'].str.contains('–')]

In [ ]:
# here is a simple Content-Based Filtering Recommendation based on popularity (rating and votes)
min_votes = movies['votes'].min()
max_votes = movies['votes'].max()

# Compute a normalized weight for each movie based on the number of votes
movies['normalized_weight'] = (movies['votes'] - min_votes) / (max_votes - min_votes)

# Compute the weighted rating
movies['weighted_rating'] = movies['rating'] * movies['normalized_weight']

# Sort movies by the weighted rating in descending order
recommended_movies = movies.sort_values(by='weighted_rating', ascending=False)

# Display top N recommended movies
top_n_recommended = recommended_movies[['title', 'rating', 'votes', 'weighted_rating']].head(15)
top_n_recommended



C:\Users\dimit\AppData\Local\Temp\ipykernel_12552\3081872888.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['normalized_weight'] = (movies['votes'] - min_votes) / (max_votes - min_votes)
C:\Users\dimit\AppData\Local\Temp\ipykernel_12552\3081872888.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['weighted_rating'] = movies['rating'] * movies['normalized_weight']


,title,rating,votes,weighted_rating
57,The_Lord_of_the_Rings:_The_Return_of_the_King,9.0,1819157.0,8.878484
20,The_Lord_of_the_Rings:_The_Fellowship_of_the_Ring,8.8,1844055.0,8.800000
78,The_Lord_of_the_Rings:_The_Two_Towers,8.8,1642708.0,7.839151
153,Gladiator,8.5,1481531.0,6.828975
173,The_Departed,8.5,1310171.0,6.039105
161,Léon:_The_Professional,8.5,1147345.0,5.288571
96,Titanic,7.9,1158746.0,4.964103
271,Kill_Bill:_Vol._1,8.2,1103348.0,4.906273
113,Jurassic_Park,8.2,978402.0,4.350671
189,Gone_Girl,8.1,971322.0,4.266515


In [631]:
###using word2vec to recommend a movie based on movies a user watch previously regarding description plot, actors and genres

# list_of_movies = movies['movie_description'].tolist()
# list_of_series = series['movie_description'].tolist()

# model_movies = Word2Vec(list_of_movies, vector_size=10, window=10, min_count=1, workers=4, sample=1e-3, sg=1, negative=5)

# model_series = Word2Vec(list_of_series, vector_size=30, window=8, min_count=1, workers=4, sample=1e-3, sg=1, negative=5)

# titles = movies.title.tolist()

# for i in range(len(titles)):
#     similarity = model_movies.wv.similarity('Bird_Box', titles[i])
#     if similarity > 0.9:
#         print(f"Bird_Box' and '{titles[i]}': {similarity}")

# not so good results

another approach using transformers - bert for description similarity
This is a recommend system base on description, genre and actors 
combination of description, genre and actors
for example if a user watch the The_Blue_Planet	the system will recommend to him to alse watch (Life_in_Colour, 0.85), (Our_Planet, 0.82)


In [ ]:
# add weighted score here!!!!!!!!!

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [633]:
movies['des'] = movies['movie_description'].apply(lambda x: ', '.join([str(i) for i in x]) if isinstance(x, list) else '')

#Encode descriptions to create embeddings
embeddings = model.encode(movies['des'].tolist())

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(embeddings)

top_n = 2
top_similar_movies = []

# Iterate over the similarity matrix to get the top 2 similar movies and their similarity scores
for i, row in enumerate(similarity_matrix):
    # Ignore self-comparison by setting similarity with itself to -1
    row[i] = -1
    # Get indices of the top 2 most similar descriptions
    top_indices = row.argsort()[-top_n:][::-1]
    
    # Retrieve movie names and similarity scores of the top 2 similar movies
    similar_movies = movies['title'].iloc[top_indices].values
    similarity_scores = row[top_indices]
    
    # Format similarity scores to 2 decimal places
    formatted_scores = [round(score, 2) for score in similarity_scores]
    
    # Append a list of tuples (movie_name, formatted_similarity_score)
    top_similar_movies.append(list(zip(similar_movies, formatted_scores)))

# Add the top similar movies along with their formatted similarity scores as a new column in the DataFrame
movies['top_similar_movies_and_scores'] = top_similar_movies

# View the DataFrame with the top similar movies and their similarity scores
movies[['title', 'top_similar_movies_and_scores']].head(40)

C:\Users\dimit\AppData\Local\Temp\ipykernel_12552\2805446100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['des'] = movies['movie_description'].apply(lambda x: ', '.join([str(i) for i in x]) if isinstance(x, list) else '')
C:\Users\dimit\AppData\Local\Temp\ipykernel_12552\2805446100.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['top_similar_movies_and_scores'] = top_similar_movies


,title,top_similar_movies_and_scores
3,Devil_in_Ohio,"[(Cuckoo_Song, 0.83), (Taboo, 0.82)]"
9,Blonde,"[(The_Lizzie_Borden_Chronicles, 0.73), (Nyad, ..."
11,End_of_the_Road,"[(Copshop, 0.82), (Rebel_Ridge, 0.79)]"
13,Glass_Onion:_A_Knives_Out_Mystery,"[(The_Lookout, 0.67), (Duisburg_-_Linea_di_san..."
14,Do_Revenge,"[(Kevin_Hart:_Zero_F**ks_Given, 0.72), (First_..."
15,I_Came_By,"[(A_Scandall, 0.71), (Selfie, 0.7)]"
16,No_Limit,"[(Along_for_the_Ride, 0.77), (Rust_and_Bone, 0..."
20,The_Lord_of_the_Rings:_The_Fellowship_of_the_Ring,"[(The_Lord_of_the_Rings:_The_Two_Towers, 0.85)..."
30,Echoes,"[(Feria:_The_Darkest_Light, 0.74), (Messiah, 0..."
31,The_Gray_Man,"[(Atomic_Blonde, 0.86), (The_Numbers_Station, ..."


In [634]:
movies[['title', 'top_similar_movies_and_scores']]

,title,top_similar_movies_and_scores
3,Devil_in_Ohio,"[(Cuckoo_Song, 0.83), (Taboo, 0.82)]"
9,Blonde,"[(The_Lizzie_Borden_Chronicles, 0.73), (Nyad, ..."
11,End_of_the_Road,"[(Copshop, 0.82), (Rebel_Ridge, 0.79)]"
13,Glass_Onion:_A_Knives_Out_Mystery,"[(The_Lookout, 0.67), (Duisburg_-_Linea_di_san..."
14,Do_Revenge,"[(Kevin_Hart:_Zero_F**ks_Given, 0.72), (First_..."
...,...,...
7892,The_Vivo_Songbook,"[(Dry_Martina, 0.62), (Ben_Platt_Live_from_Rad..."
7895,Nohzdyve,"[(Splinter_Cell, 0.73), (Horizon_Zero_Dawn, 0...."
7896,Someone_Borrowed,"[(One_Second, 0.79), (The_1st_Shop_of_Coffee_P..."
7901,Salmo_ft_Coez:_Sparare_alla_luna,[(Bump_of_Chicken_Tour_2019:_aurora_ark_Tokyo_...


In [635]:
series['movie_description'] = series['movie_description'].apply(lambda x: [x[i] for i in range(len(x)) if i not in [2, 3]])
series['des'] = series['movie_description'].apply(lambda x: ', '.join([str(i) for i in x]) if isinstance(x, list) else '')

#Encode descriptions to create embeddings
embeddings = model.encode(series['des'].tolist())

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(embeddings)

top_n = 2
top_similar_series = []

# Iterate over the similarity matrix to get the top 2 similar movies and their similarity scores
for i, row in enumerate(similarity_matrix):
    # Ignore self-comparison by setting similarity with itself to -1
    row[i] = -1
    # Get indices of the top 2 most similar descriptions
    top_indices = row.argsort()[-top_n:][::-1]
    
    # Retrieve movie names and similarity scores of the top 2 similar movies
    similar_series = series['title'].iloc[top_indices].values
    similarity_scores = row[top_indices]
    
    # Format similarity scores to 2 decimal places
    formatted_scores = [round(score, 2) for score in similarity_scores]
    
    # Append a list of tuples (movie_name, formatted_similarity_score)
    top_similar_series.append(list(zip(similar_series, formatted_scores)))

# Add the top similar movies along with their formatted similarity scores as a new column in the DataFrame
series['top_similar_movies_and_scores'] = top_similar_series

# View the DataFrame with the top similar movies and their similarity scores
series[['title', 'top_similar_movies_and_scores']].head(40)

C:\Users\dimit\AppData\Local\Temp\ipykernel_12552\342722264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series['movie_description'] = series['movie_description'].apply(lambda x: [x[i] for i in range(len(x)) if i not in [2, 3]])
C:\Users\dimit\AppData\Local\Temp\ipykernel_12552\342722264.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series['des'] = series['movie_description'].apply(lambda x: ', '.join([str(i) for i in x]) if isinstance(x, list) else '')
C:\Users\dimit\AppData\Local\Temp\ipykern

,title,top_similar_movies_and_scores
0,Cobra_Kai,"[(Haikyu!!, 0.68), (KO_One, 0.67)]"
1,The_Crown,"[(Versailles, 0.78), (Borgia, 0.75)]"
2,Better_Call_Saul,"[(The_Blacklist, 0.73), (Diamond_City, 0.72)]"
4,Cyberpunk:_Edgerunners,"[(Spriggan, 0.83), (Revisions, 0.82)]"
5,The_Sandman,"[(By_the_Grace_of_the_Gods, 0.81), (The_Order,..."
6,Rick_and_Morty,"[(Super_Monsters, 0.79), (Little_Singham, 0.79)]"
7,Breaking_Bad,"[(Elite, 0.74), (Sex_Education, 0.74)]"
8,The_Imperfects,"[(The_Witcher, 0.75), (Ada_Twist,_Scientist, 0..."
10,Stranger_Things,"[(Supernatural, 0.85), (Son:_The_Guest, 0.83)]"
12,The_Walking_Dead,"[(Slasher, 0.74), (Black_Summer, 0.72)]"


further investigation and future improvements (dataset with users_id):

user interactions with viewing history and how user rated other titles, for example because of a user watched a specific movie and voted it as five star movie, netflix recommend another same one based on genre, year, popularity, description or same on popularity
 
